<h1 style="text-align: center;">Домашнее задание №6</h1>

Необходимо подготовить отчет с результатами применения обсужденных на лекции методов к произвольным данным, которые вы найдете в свободном доступе.

1) На платформе Kaggle или на Github найдите достаточно большой датасет с логами пользования сервисов (просмотры Youtube или стриминговых сервисов, такси/каршеринг или что угодно, где есть много юзеров и история их взаимодействия с каким-нибудь сервисом). В случае, если такого датасета нет - сгенерируйте самостоятельно (например, как на лекции было показано распределение по просмотру в Кион).
2) Разбейте на сегменты вашу аудиторию и примените для нескольких (примерно 3-4 метрики) метрик тесты Стьюдента, Манна-Уитни, Фишера (Фишера для нескольких сегментов). Проинтерпретируйте результаты, сделайте первые выводы.
3) Для >=3 сегментов сделайте попарные сравнения через тест Стьюдента и тест Фишера. Проверьте соотносятся ли результаты.
4) Постройте точный и эфронов доверительные интервал для выбранных метрик. Проверьте соответствуют ли эти доверительные интервалы результатам теста Стьюдента.
5) Подумайте, можно ли использовать в пункте 2 другие стат. тесты. Приведите пару примеров.

In [1]:
# Импортируем необходимые библиотеки
from itertools import combinations
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import scipy.stats as sts
from scipy.stats import ttest_ind, mannwhitneyu, norm, shapiro, normaltest, ttest_ind_from_stats

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline

from warnings import simplefilter

## Задача 1.  Найдем достаточно большой датасет с логами пользования сервисов.

Я взял датасет, представляющий собой данные из приложения МТС Kion по взаимодействиям пользователей с контентом за период 6 месяцев (платформа ODS - https://ods.ai/competitions/competition-recsys-21). Я выбрал следующие метрики:
* total_dur - общая продолжительность всех просмотров данного контента в секундах.
* watched_pct - процент "досматриваемости" контента.
* series_cnt - количество просмотренных сериалов.

Первый и второй признаки необходимо усреднить для каждого пользователя, а третий - искусственно создать на основе данных из датасета interactions.

In [2]:
users = pd.read_csv('users.csv')
interactions = pd.read_csv('interactions.csv')
items = pd.read_csv('items.csv')

In [3]:
watched_pct = interactions[['user_id', 'watched_pct']]\
                        .groupby('user_id')\
                        .mean()

users = users.merge(watched_pct, how='inner', on=['user_id'])
users['watched_pct'] = round(users['watched_pct'], 2) 

In [4]:
series = interactions[['user_id', 'item_id']]\
                .join(items[['content_type', 'item_id']]\
                .set_index('item_id'), on='item_id')

series['content_type'] = series['content_type'] == 'series'

series = series[['user_id', 'content_type']]\
                .groupby('user_id')\
                .sum()\
                .rename(columns={'content_type': 'series_cnt'})

users = users.merge(series, how='inner', on=['user_id'])

In [5]:
total_dur = interactions[['user_id', 'total_dur']]\
                        .groupby('user_id')\
                        .mean()
users = users.merge(total_dur, how='inner', on=['user_id'])
users['total_dur'] = round(users['total_dur'], 2) 

In [6]:
df = users[['user_id', 'income', 'total_dur', 'watched_pct', 'series_cnt']]

In [7]:
df.dropna(inplace=True)

In [8]:
df

,user_id,income,total_dur,watched_pct,series_cnt
0,973171,income_60_90,53159.80,61.00,2
1,962099,income_20_40,7666.71,45.50,7
2,721985,income_20_40,3150.22,42.56,0
3,704055,income_60_90,993.40,13.90,0
4,1037719,income_60_90,8218.33,76.67,1
...,...,...,...,...,...
744281,1021167,income_20_40,103.00,0.00,1
744282,312839,income_60_90,888.00,14.00,0
744283,191349,income_40_60,3466.00,57.00,0
744284,393868,income_20_40,2819.00,13.43,5


## Задача 2.  Разбьем на сегменты нашу аудиторию и применим для нескольких метрик тесты Стьюдента, Манна-Уитни, Фишера (Фишера для нескольких сегментов).

Для теста Стьюдента и Манна-Уитни в качестве сегмента выберем income - доход пользователя: income_20_40 и income_60_90, метки метрик сложим в список. 

In [9]:
metrics = ['total_dur',
           'watched_pct',
           'series_cnt']

### Тест Стьюдента

In [10]:
for metric in metrics:
    _, p_val = sts.ttest_ind(df[df['income'] == 'income_20_40'][metric], 
                               df[df['income'] == 'income_60_90'][metric])
    print(f'{metric}: {p_val}')

total_dur: 5.6003006536731104e-108
watched_pct: 5.116590534811744e-112
series_cnt: 2.234353480393477e-37


### Тест Манна-Уитни

In [11]:
for metric in metrics:
    _, p_val = sts.mannwhitneyu(df[df['income'] == 'income_20_40'][metric], 
                                  df[df['income'] == 'income_60_90'][metric])
    print(f'{metric}: {p_val}')

total_dur: 2.02365209848869e-229
watched_pct: 6.398421466068334e-106
series_cnt: 1.5850183408606397e-51


### Тест Фишера (для нескольких сегментов)

In [12]:
for metric in metrics:
    segments = []
    for income in df['income'].unique():
        segments.append(df[df['income'] == f'{income}'][metric])
    _, p_val = sts.f_oneway(*segments)
    print(f'{metric}: {p_val}')

total_dur: 9.141376798369224e-190
watched_pct: 8.408245684879404e-248
series_cnt: 6.709140650420913e-87


### Вывод: 

* Сегменты для выбранных метрик отличаются для разных зарплатных групп.
* Тест Манна-Уитни не подходит для сравнения средних значений, поэтому и результаты между ним и тестом Стьюдента различны.
* Исходя из результатов теста Фишера для нескольких сегментов можно сказать, что хотя бы один сегмент отличается от других по всем метрикам.

## Задача 3. Для >=3 сегментов сделаем попарные сравнения через тест Стьюдента и тест Фишера.

### Тест Стьюдента

In [13]:
result = []

for pair in combinations(df['income'].unique(), 2):
    _, p_val_series_cnt = sts.ttest_ind(
        df[df['income'] == f'{pair[0]}']['series_cnt'], 
        df[df['income'] == f'{pair[1]}']['series_cnt'])
    _, p_val_watched_pct = sts.ttest_ind(
        df[df['income'] == f'{pair[0]}']['watched_pct'], 
        df[df['income'] == f'{pair[1]}']['watched_pct'])
    _, p_val_cnt_total_dur = sts.ttest_ind(
        df[df['income'] == f'{pair[0]}']['total_dur'], 
        df[df['income'] == f'{pair[1]}']['total_dur'])
    result.append({'Пара': ' - '.join(pair),
                   'Количество просмотров сериалов': p_val_series_cnt,
                   'Средний процент просмотренности контента': p_val_watched_pct,
                   'Средняя длительность просмотра': p_val_cnt_total_dur})
    
t_test = pd.DataFrame(result)
t_test

,Пара,Количество просмотров сериалов,Средний процент просмотренности контента,Средняя длительность просмотра
0,income_60_90 - income_20_40,2.234353e-37,5.116591e-112,5.600301e-108
1,income_60_90 - income_40_60,8.180710e-10,3.188261e-24,8.101391e-41
2,income_60_90 - income_90_150,1.187686e-09,2.797037e-11,3.886018e-08
3,income_60_90 - income_0_20,8.151600e-01,4.150762e-79,1.915937e-27
4,income_60_90 - income_150_inf,8.151109e-05,6.859771e-07,1.017940e-04
5,income_20_40 - income_40_60,3.159416e-24,1.176502e-82,2.129479e-43
6,income_20_40 - income_90_150,6.933876e-42,4.132237e-73,3.014753e-69
7,income_20_40 - income_0_20,8.189634e-13,6.965465e-16,8.079461e-01
8,income_20_40 - income_150_inf,2.284340e-09,4.195270e-16,2.603044e-14
9,income_40_60 - income_90_150,2.405498e-24,1.368052e-34,4.582440e-44


### Тест Фишера

In [14]:
result = []

for pair in combinations(df['income'].unique(), 2):
    _, p_val_cnt_views = sts.f_oneway(
        df[df['income'] == f'{pair[0]}']['series_cnt'], 
        df[df['income'] == f'{pair[1]}']['series_cnt'])
    
    _, p_val_mean_watched = sts.f_oneway(
        df[df['income'] == f'{pair[0]}']['watched_pct'], 
        df[df['income'] == f'{pair[1]}']['watched_pct'])
    
    _, p_val_cnt_kids_film = sts.f_oneway(
        df[df['income'] == f'{pair[0]}']['total_dur'], 
        df[df['income'] == f'{pair[1]}']['total_dur'])
    
    result.append({'Пара': ' - '.join(pair),
                   'Количество просмотров сериалов': p_val_cnt_views,
                   'Средний процент просмотренности контента': p_val_mean_watched,
                   'Средняя длительность просмотра': p_val_cnt_kids_film})
    
f_test = pd.DataFrame(result)
f_test

,Пара,Количество просмотров сериалов,Средний процент просмотренности контента,Средняя длительность просмотра
0,income_60_90 - income_20_40,2.234353e-37,5.116591e-112,5.600301e-108
1,income_60_90 - income_40_60,8.180710e-10,3.188261e-24,8.101391e-41
2,income_60_90 - income_90_150,1.187686e-09,2.797037e-11,3.886018e-08
3,income_60_90 - income_0_20,8.151600e-01,4.150762e-79,1.915937e-27
4,income_60_90 - income_150_inf,8.151109e-05,6.859771e-07,1.017940e-04
5,income_20_40 - income_40_60,3.159416e-24,1.176502e-82,2.129479e-43
6,income_20_40 - income_90_150,6.933876e-42,4.132237e-73,3.014753e-69
7,income_20_40 - income_0_20,8.189634e-13,6.965465e-16,8.079461e-01
8,income_20_40 - income_150_inf,2.284340e-09,4.195270e-16,2.603044e-14
9,income_40_60 - income_90_150,2.405498e-24,1.368052e-34,4.582440e-44


In [15]:
diff_t_f = np.abs(np.round(t_test.drop('Пара', axis=1) - f_test.drop('Пара', axis=1), 5)).sum().sum()
diff_t_f

0.0

### Вывод: 

Попарный тест Фишерра эквивалентен двухстороннему тесту Стьюдента, это можно заметить как из полученных таблиц визуально, так и численно - значение diff_t_f равно 0 в точности до 5 знака после запятой.

## Задача 4. Построим точный и эфронов доверительные интервалы для выбранных метрик.

Возьмем два сегмента дохода пользователя: income_20_40 и income_60_90. Построим для них доверительные интервалы и сравним их с результатами теста Стьюдента из 2-го пункта.

In [16]:
# Функция для расчета точного доверительного интервала:
def exact(data, alpha=0.05):
    n = len(data)
    mean = np.mean(data)
    std = np.std(data, ddof=1)
    t = sts.t.ppf(1 - alpha / 2, df=n - 1)
    lower = mean - (t * std / np.sqrt(n))
    upper = mean + (t * std / np.sqrt(n))
    return lower, upper

In [17]:
# Функция для расчета эфронова доверительного интервала:
def efron(data, alpha=0.05, n_bootstrap=1000):
    n = len(data)
    mean = np.mean(data)
    
    bootstrap_samples = np.random.choice(data, size=(n, n_bootstrap), replace=True)
    
    bootstrap_means = np.mean(bootstrap_samples, axis=0)
    
    lower_mean = np.percentile(bootstrap_means, q=alpha / 2 * 100)
    upper_mean = np.percentile(bootstrap_means, q=(1 - alpha / 2) * 100)
    
    return (lower_mean, upper_mean)

In [18]:
result = []

for metric in metrics:
    interval_20_40 = exact(df[df['income'] == 'income_20_40'][metric])
    interval_60_90 = exact(df[df['income'] == 'income_60_90'][metric])
    interval_20_40 = np.round(interval_20_40, 5)
    interval_60_90 = np.round(interval_60_90, 5)
    
    _, p_val = sts.ttest_ind(df[df['income'] == 'income_20_40'][metric], 
                               df[df['income'] == 'income_60_90'][metric])
    p_val = np.round(p_val, 5)

    print(f'{metric}: {interval_20_40}, {interval_60_90}, {p_val}')

total_dur: [7103.3479  7235.88603], [9094.8356  9496.95839], 0.0
watched_pct: [37.52449 37.71015], [40.34445 40.83473], 0.0
series_cnt: [1.28335 1.29767], [1.40174 1.44133], 0.0


In [19]:
for metric in metrics:
    interval_20_40 = efron(df[df['income'] == 'income_20_40'][metric])
    interval_60_90 = efron(df[df['income'] == 'income_60_90'][metric])
    interval_20_40 = np.round(interval_20_40, 5)
    interval_60_90 = np.round(interval_60_90, 5)
    
    _, p_val = sts.ttest_ind(df[df['income'] == 'income_20_40'][metric], 
                               df[df['income'] == 'income_60_90'][metric])
    p_val = np.round(p_val, 5)
    
    print(f'{metric}: {interval_20_40}, {interval_60_90}, {p_val}')

total_dur: [7101.92323 7237.41414], [9098.92729 9489.1581 ], 0.0
watched_pct: [37.53013 37.71052], [40.38026 40.83143], 0.0
series_cnt: [1.28385 1.29716], [1.4034  1.44279], 0.0


### Вывод: 

* И точный, и эфронов доверительные интервалы показали одинаковые результаты.
* Построенные доверительные интервалы подтверждают полученные выше значения теста Стьюдента.
* Использование доверительных интервалов считается менее точным, поскольку может вызывать ошибку 2-го рода.

## Задача 5. Используем в пункте 2 другие стат. тесты.

### Тест Краскела — Уоллиса

Тест Краскела — Уоллиса предназначен для проверки равенства медиан нескольких выборок. Данный критерий является многомерным обобщением критерия Уилкоксона — Манна — Уитни. Критерий Краскела — Уоллиса является ранговым, поэтому он инвариантен по отношению к любому монотонному преобразованию шкалы измерения.

In [20]:
_, p_val = sts.kruskal(df.query("income == 'income_20_40'")['series_cnt'], 
                          df.query("income == 'income_60_90'")['series_cnt'])
p_val

1.5850179438314317e-51

*Вывод: поскольку p < 0.05, следовательно медианы распределений не равны*

### Тест Уилкоксона

Тест Уилкоксона проверяет нулевую гипотезу о том, что две связанные парные выборки получены из одного и того же распределения. В частности, он проверяет, симметрично ли распределение различий x - y относительно нуля. Это непараметрическая версия парного T-критерия.

In [21]:
_, p_val = sts.wilcoxon(df.query("income == 'income_20_40'")['series_cnt'].sample(30000), 
                          df.query("income == 'income_60_90'")['series_cnt'].sample(30000))
p_val

3.842290438005416e-12

*Вывод: поскольку p < 0.05, следовательно выборки получены из различных распределений*

### Тест Колмогорова-Смирнова

Тест Колмогорова-Смирнова используется для проверки гипотезы о принадлежности двух независимых выборок одному закону распределения, то есть о том, что два эмпирических распределения соответствуют одному и тому же закону.

In [22]:
_, p_val = sts.ks_2samp(df.query("income == 'income_20_40'")['series_cnt'], 
                          df.query("income == 'income_60_90'")['series_cnt'])
p_val

3.9938986160082435e-33

*Вывод: поскольку p < 0.05, следовательно распределения различны*